# 1. Devices pre-processing

This task will fetch the raw data from the remote database given a SQL query file.

In [1]:
import sys
sys.path.append('..')

## 1.1 Import libraries

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import MySQLdb
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

In [4]:
DATA_DIR = '../../data'
STORAGE_DIR = os.path.join(DATA_DIR, 'storage')

if not os.path.exists(STORAGE_DIR):
    os.makedirs(STORAGE_DIR)

## 1.2 Create database connection
Create a database connection using the driver `mysqlclient`

In [5]:
model = 'devices'
connection_string = 'mysql://{}:{}@{}/{}'.format(os.getenv('DB_USERNAME'), os.getenv('DB_PASSWORD'), \
                                                 os.getenv('DB_HOST'), os.getenv('DB_DATABASE'))
engine = create_engine(connection_string, echo=True)

### 1.2.1 Read the SQL query file

In [6]:
sql_query = open(os.path.join(DATA_DIR, 'sql/devices.sql')).read()

## 1.3 Fetch the remote data

In [7]:
df = pd.read_sql(sql_query, engine)

2020-07-31 00:15:34,824 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-07-31 00:15:34,825 INFO sqlalchemy.engine.base.Engine ()
2020-07-31 00:15:34,849 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-07-31 00:15:34,850 INFO sqlalchemy.engine.base.Engine ()
2020-07-31 00:15:34,891 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-07-31 00:15:34,892 INFO sqlalchemy.engine.base.Engine ()
2020-07-31 00:15:34,931 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-07-31 00:15:34,932 INFO sqlalchemy.engine.base.Engine ()
2020-07-31 00:15:34,955 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-07-31 00:15:34,956 INFO sqlalchemy.engine.base.Engine ()
2020-07-31 00:15:34,981 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-07-31 00:15:34,982 INFO sqlalchemy.engine.base.E

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306469 entries, 0 to 306468
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            306469 non-null  int64 
 1   model         306469 non-null  object
 2   manufacturer  306469 non-null  object
 3   brand         306469 non-null  object
 4   os_version    306469 non-null  object
 5   is_root       306469 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 14.0+ MB


## 1.4 Basic data pre-processing
Benchmarking string operations

In [9]:
string_columns = df.select_dtypes(include='object').columns.to_list()

In [10]:
for column in string_columns:
    df[column] = [x.lower().strip() for x in df[column].tolist()]

In [11]:
mappings = {'id':'uint32', 'model':'category', 'manufacturer':'category',
            'brand':'category', 'os_version':'category', 'is_root': 'uint8'}

In [12]:
df = df.astype(mappings)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306469 entries, 0 to 306468
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            306469 non-null  uint32  
 1   model         306469 non-null  category
 2   manufacturer  306469 non-null  category
 3   brand         306469 non-null  category
 4   os_version    306469 non-null  category
 5   is_root       306469 non-null  uint8   
dtypes: category(4), uint32(1), uint8(1)
memory usage: 4.4 MB


## 1.5 Save output
Storage data in parquet format with brotli compression.

In [13]:
df.to_parquet(os.path.join(STORAGE_DIR, '{}.parquet'.format(model)), compression='brotli')